# Gene Set Enrichment - ORA - SetRank

In [1]:
library("gage")
library("plyr")
library("SetRank")
library("GeneSets.Homo.sapiens")
library("tidyverse")
#library("parallel")
library("AnnotationDbi")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterA

In [2]:
options(mc.cores=20)

## Load Gene sets

In [3]:
datDir <- "../results/DEG_overlaps/"

In [4]:
deg_res_sp1 <- readRDS(file.path(datDir, "deg_res_sp1.rds")) # DC
deg_res_sp2 <- readRDS(file.path(datDir, "deg_res_sp2.rds")) # AFU
deg_res_sp3 <- readRDS(file.path(datDir, "deg_res_sp3.rds")) # CMV

In [5]:
l <- list.files(datDir, "*.rds", full.names = T)
l <- grep("deg_res", l, value = T, invert = T)
gene_sets <- lapply(l, function(x) readRDS(x))

In [6]:
cond <- basename(l)
cond <- sub("\\.rds", "", cond)
cond

[1] "AFU-SingleCulture_Afu_0h"            "AFU-SingleCulture_Afu_4.5h"         
 [3] "AFU-SingleCulture_Afu-all"           "CMV-coinfection_VS_singleinfection" 
 [5] "DC-AFu_vs_CMV_vs_Both"               "DC-All_AFu"                         
 [7] "DC-All_CMV"                          "DC-All_intersect"                   
 [9] "DC-SingleCulture_DC-all"             "DC-SingleCulture_DC-Coinfection"    
[11] "DC-SingleCulture_DC-SingleInfection"

In [7]:
names(gene_sets) <- cond

## Build Background

In [8]:
refset_from_deg_tables <- function(df) sapply(df$DEGs, rownames) %>% unlist %>% unique

In [9]:
calc_background <- function(refGeneSet, saveFile, maxSetSize = 500) {
    collection = buildSetCollection(BIOCYC, GOMF, GOBP, KEGG, referenceSet = refGeneSet, maxSetSize = 500)
    save(collection, file = saveFile)
    return(collection)
}

In [10]:
dc.refGeneSet <- refset_from_deg_tables(deg_res_sp1)
afu.refGeneSet <- refset_from_deg_tables(deg_res_sp2)
cmv.refGeneSet <- refset_from_deg_tables(deg_res_sp3)

In [ ]:
# Convert to entrez identifiers
dc.ensembl2EntrezID = createIDConverter("org.Hs.eg.db", "ENSEMBL", "ENTREZID")
dc.refGeneSet.entrez = dc.ensembl2EntrezID(dc.refGeneSet)
head(dc.refGeneSet.entrez)

In [28]:
#head(allDBs, n = 1)

In [14]:
ls("package:GeneSets.Homo.sapiens")

[1] "allDBs"                       "BIOCYC"                      
[3] "GOBP"                         "GOCC"                        
[5] "GOMF"                         "KEGG"                        
[7] "Pathway_Interaction_Database" "REACTOME"                    
[9] "WikiPathways"

In [13]:
#dc.collection <- calc_background(dc.refGeneSet.entrez, "../results/GeneSetEnrichment/dc.collection.500.rda")
load("../results/GeneSetEnrichment/dc.collection.500.rda")
dc.collection <- collection

## Calc RankSets

In [68]:
calc_rank_set <- function(geneSet,  idconverter.in, collection) {
    # remove terminal '-', '+' symbols and convert to entrez
    geneSet <- geneSet %>% gsub(".$", "", .) %>% idconverter.in
    rankSet <- setRankAnalysis(geneSet, collection, use.ranks = T, setPCutoff = 0.01, fdrCutoff = 0.05)
    
    return(list(rankSet = rankSet, geneSet = geneSet))
}

write_out_rankset <- function(l, outDir, pref, idconverter.out, collection) {
    options(warn = -1)
    exportSingleResult(l$rankSet, l$geneSet, collection, pref, IDConverter = idconverter.out, outputPath = outDir)
    options(warn = 0)
}

In [16]:
is_dc <- gene_sets %>% names %>% grepl("DC", .)
dc.gene_sets <- gene_sets[is_dc]

In [24]:
names(dc.gene_sets)

[1] "DC-AFu_vs_CMV_vs_Both"               "DC-All_AFu"                         
[3] "DC-All_CMV"                          "DC-All_intersect"                   
[5] "DC-SingleCulture_DC-all"             "DC-SingleCulture_DC-Coinfection"    
[7] "DC-SingleCulture_DC-SingleInfection"

Define all sets of interest

In [60]:
dc.sets_of_interest <- list()
# Which are present regardless of pathogen and growth
item <- "SingleInfection_CMV_0h:SingleInfection_Afu_4h30min:SingleInfection_Afu_0h:SingleInfection_CMV_2h:CoInfection_CMV_first:CoInfection:CoInfection_Afu_first"
dc.sets_of_interest$intersect <- dc.gene_sets[['DC-SingleCulture_DC-all']][[item]]

# Which is specific for single infection CMV - independent of time
item <- dc.gene_sets[['DC-SingleCulture_DC-all']] %>% {names(.) == 'SingleInfection_CMV_0h:SingleInfection_CMV_2h'} %>% which
names(dc.gene_sets[['DC-SingleCulture_DC-all']])[[item]]
dc.sets_of_interest$si_cmv <- dc.gene_sets[['DC-SingleCulture_DC-all']][[item]]

# Which is specific for signle infection AFU - independent of time
item <- dc.gene_sets[['DC-SingleCulture_DC-all']] %>% {names(.) == 'SingleInfection_Afu_4h30min:SingleInfection_Afu_0h'} %>% which
names(dc.gene_sets[['DC-SingleCulture_DC-all']])[[item]]
dc.sets_of_interest$si_afu <- dc.gene_sets[['DC-SingleCulture_DC-all']][[item]]

# Which is specific for AFU - independent of time and single/co
item <- dc.gene_sets[['DC-SingleCulture_DC-all']] %>% {names(.) == 'SingleInfection_Afu_4h30min:SingleInfection_Afu_0h:CoInfection_CMV_first:CoInfection:CoInfection_Afu_first'} %>% which
names(dc.gene_sets[['DC-SingleCulture_DC-all']])[[item]]
dc.sets_of_interest$si_co_afu <- dc.gene_sets[['DC-SingleCulture_DC-all']][[item]]

# Which is specific for CMV - independent of time and single/co
item <- dc.gene_sets[['DC-SingleCulture_DC-all']] %>% {names(.) == 'SingleInfection_CMV_0h:SingleInfection_CMV_2h:CoInfection_CMV_first:CoInfection:CoInfection_Afu_first'} %>% which
names(dc.gene_sets[['DC-SingleCulture_DC-all']])[[item]]
dc.sets_of_interest$si_co_cmv <- dc.gene_sets[['DC-SingleCulture_DC-all']][[item]]

# Which is specific for coinfection - independent of type and time
item <- dc.gene_sets[['DC-SingleCulture_DC-all']] %>% {names(.) == 'CoInfection_CMV_first:CoInfection:CoInfection_Afu_first'} %>% which
names(dc.gene_sets[['DC-SingleCulture_DC-all']])[[item]]
dc.sets_of_interest$co_union <- dc.gene_sets[['DC-SingleCulture_DC-all']][[item]]

[1] "SingleInfection_CMV_0h:SingleInfection_CMV_2h"

[1] "SingleInfection_Afu_4h30min:SingleInfection_Afu_0h"

[1] "SingleInfection_Afu_4h30min:SingleInfection_Afu_0h:CoInfection_CMV_first:CoInfection:CoInfection_Afu_first"

[1] "SingleInfection_CMV_0h:SingleInfection_CMV_2h:CoInfection_CMV_first:CoInfection:CoInfection_Afu_first"

[1] "CoInfection_CMV_first:CoInfection:CoInfection_Afu_first"

In [ ]:
dc.entrezID2ensembl = createIDConverter("org.Hs.eg.db", "ENTREZID", "ENSEMBL")
dc.entrezID2symbol = createIDConverter("org.Hs.eg.db", "ENTREZID", "SYMBOL")

In [ ]:
dc.ranksets_of_interest <- lapply(1:length(dc.sets_of_interest), function(i) {
    print(paste0("Calc: ", names(dc.sets_of_interest)[i]))
    res <- calc_rank_set(dc.sets_of_interest[[i]],
                          dc.ensembl2EntrezID,
                          dc.collection)
    return(res)
})

In [ ]:
saveRDS(dc.ranksets_of_interest, "../results/GeneSetEnrichment/dc.ranksets_of_interest.rds")

In [ ]:
# write out 2:length(dc.ranksets_of_interest)
for (i in 1:length(dc.ranksets_of_interest)) {
    write_out_rankset(l      = dc.ranksets_of_interest[[i]],
                      outDir = paste0("../results/GeneSetEnrichment/DC-", names(dc.sets_of_interest)[[i]]),
                      pref   = paste0("DC-", names(dc.sets_of_interest)[i]),
                      idconverter.out = dc.entrezID2symbol,
                      collection = dc.collection)
}

#for (i in 1:length(dc.ranksets_of_interest)) {
#    write_out_rankset(l      = dc.ranksets_of_interest[[i]],
#                      outDir = paste0("../results/GeneSetEnrichment/DC-", names(dc.sets_of_interest)[[i]]),
#                      pref   = paste0("DC-", names(dc.sets_of_interest)[i]),
#                      idconverter.out = NULL,
#                      collection = dc.collection)
#}

# Score Based - GAGE

In [1]:
library("gage")
library("plyr")
library("AnnotationDbi")
library("tidyverse")

Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.min

Loading required package: Biob

## Load Data

In [2]:
data(kegg.gs)

In [25]:
kegg.gs.ez <- kegg.gsets(species = "hsa", id.type = "entrez", check.new = T)

In [11]:
head(deg_res_sp1$DEGs$CoInfection_CMV_first_vs_SingleInfection_CMV_0h)

,id,mean_A_mrn,mean_B_mrn,log2_fc_mrn,mean_A_tpm,mean_B_tpm,log2_fc_tpm,mean_A_rpkm,mean_B_rpkm,fc_rpkm,log2_fc_rpkm,DESeq,DESeq_adj_pval,DESeq2,DESeq2_adj_pval,Limma,Limma_adj_pval,EdgeR,EdgeR_adj_pval
ENSG00000223972,ENSG00000223972,1.000000,1.199687,-0.26265771,1.000000,1.018014,-0.02575758,1.000000,1.023355,0.9771776,-0.033307329,FALSE,1,FALSE,1,FALSE,0.9874010,FALSE,1
ENSG00000227232,ENSG00000227232,1.255064,2.966155,-1.24083310,1.037465,1.233370,-0.24954211,1.028688,1.267397,0.8116544,-0.301062466,FALSE,1,FALSE,1,FALSE,0.8208995,FALSE,1
ENSG00000278267,ENSG00000278267,1.582591,2.309256,-0.54513968,2.056217,4.026953,-0.96969638,2.626953,4.568267,0.5750437,-0.798256599,FALSE,1,FALSE,1,FALSE,0.8894456,FALSE,1
ENSG00000268903,ENSG00000268903,1.529228,1.000000,0.61280314,1.046827,1.000000,0.06602323,1.076426,1.000000,1.0764256,0.106248558,FALSE,1,FALSE,1,FALSE,0.5106635,FALSE,1
ENSG00000241860,ENSG00000241860,1.343090,1.000000,0.42555599,1.013434,1.000000,0.01925248,1.012234,1.000000,1.0122344,0.017543404,FALSE,1,FALSE,1,FALSE,0.5106635,FALSE,1
ENSG00000279928,ENSG00000279928,1.171545,1.199687,-0.03424534,1.073005,1.054832,0.02464223,1.066484,1.071091,0.9956993,-0.006217947,FALSE,1,FALSE,1,FALSE,0.8894456,FALSE,1


In [27]:
grep("lectin", names(kegg.gs.ez$kg.sets), value = T)

[1] "hsa04966 Collecting duct acid secretion"

In [8]:
datDir <- "../results/DEG_overlaps/"

In [9]:
deg_res_sp1 <- readRDS(file.path(datDir, "deg_res_sp1.rds")) # DC

In [10]:
names(deg_res_sp1$DEGs)

[1] "CoInfection_CMV_first_vs_SingleInfection_CMV_0h"     
 [2] "CoInfection_CMV_first_vs_SingleInfection_Afu_4h30min"
 [3] "CoInfection_vs_SingleInfection_CMV_0h"               
 [4] "CoInfection_vs_SingleInfection_Afu_0h"               
 [5] "CoInfection_Afu_first_vs_SingleInfection_CMV_2h"     
 [6] "CoInfection_Afu_first_vs_SingleInfection_Afu_0h"     
 [7] "CoInfection_CMV_first_vs_CoInfection"                
 [8] "CoInfection_Afu_first_vs_CoInfection"                
 [9] "Coinfection_Afu_first_vs_CoInfection_CMV_first"      
[10] "SingleInfection_CMV_0h_vs_SingleCulture_DC"          
[11] "SingleInfection_Afu_4h30min_vs_SingleCulture_DC"     
[12] "SingleInfection_Afu_0h_vs_SingleCulture_DC"          
[13] "SingleInfection_CMV_2h_vs_SingleCulture_DC"          
[14] "CoInfection_CMV_first_vs_SingleCulture_DC"           
[15] "CoInfection_vs_SingleCulture_DC"                     
[16] "CoInfection_Afu_first_vs_SingleCulture_DC"

In [14]:
# extract MRN fold change and stuff
gene_lists <- llply(deg_res_sp1$DEGs, function(df) {
    l <- df$log2_fc_mrn
    names(l) <- rownames(df)
    return(l)
})

In [67]:
columns(org.Hs.eg.db)

[1] "ACCNUM"       "ALIAS"        "ENSEMBL"      "ENSEMBLPROT"  "ENSEMBLTRANS"
 [6] "ENTREZID"     "ENZYME"       "EVIDENCE"     "EVIDENCEALL"  "GENENAME"    
[11] "GO"           "GOALL"        "IPI"          "MAP"          "OMIM"        
[16] "ONTOLOGY"     "ONTOLOGYALL"  "PATH"         "PFAM"         "PMID"        
[21] "PROSITE"      "REFSEQ"       "SYMBOL"       "UCSCKG"       "UNIGENE"     
[26] "UNIPROT"

In [68]:
query <- names(gene_lists$CoInfection_CMV_first_vs_SingleInfection_CMV_0h)[1:10]

In [70]:
enz_ids <- mapIds(org.Hs.eg.db, keys = query, keytype = "ENSEMBL", column = "ENTREZID")
enz_ids

'select()' returned 1:1 mapping between keys and columns


ENSG00000223972 ENSG00000227232 ENSG00000278267 ENSG00000268903 ENSG00000241860 
    "100287102"              NA     "102466751"              NA              NA 
ENSG00000279928 ENSG00000279457 ENSG00000228463 ENSG00000236679 ENSG00000237094 
             NA              NA              NA              NA              NA

In [146]:
select_genes_by_pathway <- function(pw, fc, map) {
    if (length(pw) > 0) {
        sig_map <- map[pw]
        df <- ldply(sig_map, .id = "pathway", function(l) {
            inds <- (fc %>% names) %in% l
            return(data.frame(gene = names(fc)[inds], mrn_fc = fc[inds]))
        })
        df$gene <- mapIds(org.Hs.eg.db, keys = df$gene %>% as.character, keytype = "ENTREZID", column = "SYMBOL")
        return(df)
    } else
        return(data.frame())
}

enrich_patchway_mrn <- function(gene_fc_list, same.dir = T) {
    # convert to entrez gene ids
    ens_ids <- gene_fc_list %>% names
    enz_ids <- mapIds(org.Hs.eg.db, keys = ens_ids, keytype = "ENSEMBL", column = "ENTREZID")
    keep <- !is.na(enz_ids)
    gene_fc_list <- gene_fc_list[keep]
    names(gene_fc_list) <- enz_ids[keep]
    
    # perform enrichment
    fc.kegg.p <- gage(gene_fc_list, gsets = kegg.gs, ref = NULL, samp = NULL, same.dir = same.dir)
    gr <- fc.kegg.p$greater %>% as.data.frame %>% mutate(pw_name = rownames(.)) %>% dplyr::select(pw_name, everything()) %>%
        filter(q.val < 0.1 & !is.na(q.val))
    grGenes <- select_genes_by_pathway(gr$pw_name, gene_fc_list, kegg.gs)
    if (same.dir) {
        le <- fc.kegg.p$less %>% as.data.frame %>% mutate(pw_name = rownames(.)) %>% dplyr::select(pw_name, everything()) %>% 
            filter(q.val < 0.1 & !is.na(q.val))
        leGenes <- select_genes_by_pathway(le$pw_name, gene_fc_list, kegg.gs)
        return(list(grS = gr, leS = le, grGenes = grGenes, leGenes = leGenes, gr = fc.kegg.p$greater, le = fc.kegg.p$less, stats = fc.kegg.p$stats))
    } else
        return(list(grS = gr, grGenes = grGenes, gr = fc.kegg.p$greater))
}

In [154]:
gage.kegg.dir <- lapply(gene_lists, function(l) enrich_patchway_mrn(l, T))

'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()'

In [155]:
gage.kegg.both <- lapply(gene_lists, function(l) enrich_patchway_mrn(l, F))

'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'

In [163]:
dir.create("../results/GeneSetEnrichment/GAGE-DC/unidir", recursive = T)
dir.create("../results/GeneSetEnrichment/GAGE-DC/bidir", recursive = T)

In [164]:
for (i in 1:length(gage.kegg.dir))
    writexl::write_xlsx(sapply(gage.kegg.dir[[i]], function(x) x %>% as.data.frame %>% mutate(rownames = rownames(.)) %>% dplyr::select(rownames, everything())),
                        paste0("../results/GeneSetEnrichment/GAGE-DC/unidir/gage.kegg.dir_", names(gage.kegg.dir)[i], ".xlsx"))

In [165]:
for (i in 1:length(gage.kegg.dir))
    writexl::write_xlsx(sapply(gage.kegg.both[[i]], function(x) x %>% as.data.frame %>% mutate(rownames = rownames(.)) %>% dplyr::select(rownames, everything())),
                        paste0("../results/GeneSetEnrichment/GAGE-DC/bidir/gage.kegg.dir_", names(gage.kegg.both)[i], ".xlsx"))